In [ ]:
import numpy as np
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [ ]:
# Single Gaussian:
def gauss(x, p):
    return p[0] * np.exp(- (x - p[1])*(x - p[1]) / (2 * p[2] * p[2]))
    # p[0] = prefactor; p[1] = mu; p[2] = sigma

# Probability distribution:
def P(x0, y0, p1, p2, t):
    
    # translate max. to origin:
    x = x0 - p1[1]
    y = y0 - p2[1]
    
    # rotate coordinates:
    xr = np.cos(t) * x - np.sin(t) * y
    yr = np.sin(t) * x + np.cos(t) * y
    
    # translate back:
    xr += p1[1]
    yr += p2[1]
    
    # output the 2d gaussian:
    return (gauss(xr, p1)*gauss(yr, p2))

# Periodic boundary conditions:
def pbc(x, xm, ym):
    while x[0] > xm:
        x[0] -= xm
    while x[0] < 0:
        x[0] += xm

    while x[1] > ym:
        x[1] -= ym
    while x[1] < 0:
        x[1] += ym

In [ ]:
# Distribution params:
p1 = [1, 7, 0.75]
p2 = [1, 5, 3]
rot = np.pi/4

xmax = 14
ymax = 10

# Final histogram bins:
Hbins = 20

x_e = np.linspace(0,xmax,Hbins)
y_e = np.linspace(0,ymax,Hbins) # edges

X_e, Y_e = np.meshgrid(x_e, y_e) # edges

# Bins/histogram for initial distr plotting:
x_hr = np.linspace(0,xmax,200)
y_hr = np.linspace(0,ymax,200)

X_hr, Y_hr= np.meshgrid(x_hr, y_hr)

Z = P(X_hr, Y_hr, p1, p2, rot)

# Initialize Simulation:
Nsteps = 20000

# How far the random walk tries to jump:
#jsig = 7.5 # too large
#jsig = 0.075 # too small 
jsig = 0.75 # just right

xc = np.array([1, 1], dtype=float)
xp = np.array([1, 1], dtype=float)
x = np.array([xc, xc], dtype=float) # initial pos.
P_curr = P(xc[0], xc[1], p1, p2, rot)


In [ ]:
# Initialize plot
plt.ion()
fig = plt.figure(figsize=(12,8))
fig.canvas.draw()

ax1 = fig.add_subplot(221)
ax1.set_title('Original Probability Distribution')
ax1.pcolor(X_hr,Y_hr,Z)

ax2 = fig.add_subplot(222)
ax2.set_xlim([0, xmax])
ax2.set_ylim([0, ymax])
ax2.set_title('Markov Chain')

ax3 = fig.add_subplot(223)
ax3.set_title('MCMC Histogram')

ms = 4
tail = 30

line1, = ax2.plot(xc[0], xc[1], color='darkgray', marker='o', markersize=ms, linestyle='-')
line2, = ax2.plot(xc[0], xc[1], color='darkblue', marker='o', markersize=ms,  linestyle='-')
line3, = ax2.plot(xc[0], xc[1], color='red', marker='o', markersize=ms,  linestyle='-')
N_rej = 0



for step in range(Nsteps):
    
    # Propose new x:
    theta = np.random.uniform(0, np.pi) # angle
    r = np.random.normal(0, scale=jsig) # distance
    xp = np.array([r * np.cos(theta), r * np.sin(theta)], dtype=float) + xc[:]
    pbc(xp, xmax, ymax)

    # Calculate the value of P at new x:
    P_prop = P(xp[0], xp[1], p1, p2, rot)

    # Monte Carlo step:
    if P_prop >= P_curr:
        xc = xp
        P_curr = P_prop
        line3.set_xdata([])
        line3.set_ydata([])
    else:
        if np.random.uniform(0, 1) < (P_prop / P_curr):
            xc = xp
            P_curr = P_prop
            line3.set_xdata([])
            line3.set_ydata([])
        else:
            line3.set_xdata([xc[0], xp[0]])
            line3.set_ydata([xc[1], xp[1]])
            N_rej += 1

    # Add state to Markov chain:
    x = np.vstack([x, xc]) # accept
    
    # Update plots:
    line1.set_xdata(x[1:,0])
    line1.set_ydata(x[1:,1])
    
    line2.set_xdata(x[-tail:,0])
    line2.set_ydata(x[-tail:,1])
    
    if step%100 == 0:
        H = np.histogram2d(x[1:,0], x[1:,1], bins=(x_e, y_e))[0].T
        ax3.pcolormesh(X_e, Y_e, H)
        ax3.set_title('MCMC Histogram: $N = {0}$'.format(step))

    fig.show()
    fig.canvas.draw()

H = np.histogram2d(x[1:,0], x[1:,1], bins=(x_e, y_e))[0].T
ax3.pcolormesh(X_e, Y_e, H)
ax3.set_title('MCMC Histogram: $N = {0}$'.format(Nsteps))

print('Final rejection rate = {0:0.5f}'.format(N_rej / Nsteps))

In [ ]:
fig.savefig('longsim_hist2.pdf')